In [2]:
import triton
import torch
import triton.language as tl
from triton.runtime import driver

In [4]:
DEVICE = triton.runtime.driver.active.get_active_torch_device()
DEVICE

device(type='cuda', index=0)

x: a x m
y: m x n
z: a x n

In [25]:
def matmul(x, y, z, x_row, y_col):
    assert z.shape[0] == x_row and z.shape[1] == y_col
    for row in range(x_row):
        for col in range(y_col):
            z[row, col] = (x[row, :] * y[:, col]).sum()
    
    return z

In [32]:
a, m, n = 3, 3, 3
x = torch.ones((a, m))
y = torch.ones((m, n))
z = torch.zeros((a, n))

%timeit matmul(x, y, z, a, n)

148 μs ± 2.4 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [33]:
x = torch.ones((a, m), device=DEVICE)
y = torch.ones((m, n), device=DEVICE)
z = torch.zeros((a, n), device=DEVICE)

%timeit matmul(x, y, z, a, n)

515 μs ± 17.5 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


why is it 5x slower on the gpu?? BECAUSE for loops run on CPU backend, and then tensors are on GPU. GPU/CPU sync makes it slower.